In [621]:
from tensorflow.keras import layers
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.utils import plot_model, image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow import convert_to_tensor
import tensorflow.keras.backend as K

import tensorflow as tf

tf.compat.v1.enable_eager_execution()
AUTOTUNE = tf.data.AUTOTUNE

In [622]:
import pandas as pd
import numpy as np
import pathlib
import cv2
import os

### Paths

In [623]:
images_path = '../../../Dataset/img_celeba_cropped'
identity_path = '../../../Dataset/identity_CelebA.txt'
attributes_path = '../../../Dataset/list_attr_celeba.txt'

### Hyperparameters

In [663]:
images_num = 50000
image_shape = (224, 224, 3)

batch_size = 32
lr=0.001

### Creating pairs

In [673]:
celeb_identity = pd.read_csv(identity_path, sep = " ", names=["image", "identity"])[:images_num-1]
celeb_identity["identity"] = celeb_identity["identity"].astype("string")

celeb_attrs = pd.read_csv(attributes_path, sep = "\s+")[:images_num-1]

In [674]:
def get_balanced_pair(column:str, pairing_column:str, df=celeb_identity):
    # getting random identity from the provided column to use for balanced pair
    random_id = df[column].sample(1, replace=True).to_string(index=False)
    containing_id_list = df.loc[df[column] == random_id][pairing_column].sample(2, replace=True).to_list()
    
    # random pictures for pair generation
    df_without_id = df.loc[df[pairing_column] != random_id]
    negative = df[pairing_column].sample(1, replace=True).to_list()
    
    return [[containing_id_list[0], containing_id_list[1], 1], [containing_id_list[0], negative[0], 0]]

In [675]:
balanced_pairs = []

for record in range(10000):
    pair = get_balanced_pair("identity", "image")
    balanced_pairs.append(pair[0])
    balanced_pairs.append(pair[1])
    
df = pd.DataFrame(balanced_pairs, columns =['left', 'right', 'label'])
df['left'] = images_path + '/' + df['left']
df['right'] = images_path + '/' + df['right']

In [677]:
df.head()

,left,right,label
0,../../../Dataset/img_celeba_cropped/002084.jpg,../../../Dataset/img_celeba_cropped/034289.jpg,1
1,../../../Dataset/img_celeba_cropped/002084.jpg,../../../Dataset/img_celeba_cropped/022060.jpg,0
2,../../../Dataset/img_celeba_cropped/035540.jpg,../../../Dataset/img_celeba_cropped/021204.jpg,1
3,../../../Dataset/img_celeba_cropped/035540.jpg,../../../Dataset/img_celeba_cropped/042820.jpg,0
4,../../../Dataset/img_celeba_cropped/002545.jpg,../../../Dataset/img_celeba_cropped/013740.jpg,1


### Loading pairs into TF Dataset

In [683]:
dataset = tf.data.Dataset.from_tensor_slices((df['left'].values, df['right'].values, df['label'].values))

In [684]:
def preprocessing(left, right, label):
    left_image = convert_to_img(left)
    right_image = convert_to_img(right)
    
    return left_image, right_image, label


def convert_to_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img)
    img = resnet50.preprocess_input(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.reshape(img, (224,224))
    return img

In [685]:
dataset = dataset.map(preprocessing)

dataset.batch(batch_size, num_parallel_calls=AUTOTUNE)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)

<TakeDataset element_spec=(TensorSpec(shape=(224, 224), dtype=tf.float32, name=None), TensorSpec(shape=(224, 224), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [651]:
train_dataset = dataset.take(round(images_num * 0.8))
val_dataset = dataset.skip(round(images_num * 0.8))

### Input layers

In [655]:
left_input = layers.Input(shape=image_shape, name='left_input', dtype=tf.float32)
right_input = layers.Input(shape=image_shape, name='right_input', dtype=tf.float32)

### Resnet model

In [656]:
resnet=ResNet50(
    include_top=False,
    input_shape=input_shape,
    pooling='avg',
    weights='imagenet',
    classes=2)

for layer in resnet.layers[0:-1]:
    layer.trainable = False
    
left_resnet = resnet(left_input)
right_resnet = resnet(right_input)

### Merging resnet branches layers

In [657]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.reduce_sum(tf.square(subtract(x, y)), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, K.epsilon()))

In [691]:
merged = layers.subtract([left_resnet, right_resnet])
#merged = layers.Lambda(euclidean_distance, name="merge")([left_resnet, right_resnet])

flatten = layers.Flatten()(merged)

merged_dense_1 = layers.Dense(256, activation='relu')(flatten)
merged_dense_2 = layers.Dense(32, activation='relu')(merged_dense_1)

merged_output = layers.Dense(1, activation='sigmoid')(merged_dense_2)

model = Model(inputs = [left_input, right_input], outputs = merged_output)

In [692]:
# margin is a parametr settable by developer
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        '''Contrastive loss from Hadsell-et-al.'06
        http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
        '''
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - y_pred, 0))
        return (y_true * square_pred + (1 - y_true) * margin_square)    
    return contrastive_loss

In [694]:
model.compile(optimizer=Adam(learning_rate=lr), loss=contrastive_loss_with_margin(margin=1.0), metrics=['accuracy'])

In [695]:
#plot_model(resnet, show_shapes=True)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 left_input (InputLayer)        [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 right_input (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 2048)         23587712    ['left_input[0][0]',             
                                                                  'right_input[0][0]']      

In [696]:
model_fit = model.fit(inpust = train_dataset, epochs=5, validation_data=val_dataset)

Epoch 1/5


ValueError: in user code:

    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/igor/miniforge3/envs/my_env/lib/python3.8/site-packages/k

### Performance metrics

In [ ]:
plt.plot(combo_model_fitted.history['accuracy'])
plt.plot(combo_model_fitted.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.axis(ymin=0.5,ymax=1)
plt.grid()

plt.show()

In [ ]:
plt.plot(combo_model_fitted.history['loss'])
plt.plot(combo_model_fitted.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.axis(ymin=0,ymax=0.5)
plt.grid()

plt.show()

### Predictions

In [ ]:
left_image_feature_vectors = model.predict(custom_img_left)
left_image_feature_vectors = model.predict(custom_img_right)